### Bronze to Silver (Transform Bronze Data)

In [1]:
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("BronzeToSilver")
    .master("local[*]")
    .getOrCreate()
)

spark

In [33]:
raw_data_lake_schema = '''
  _row_id BIGINT,
  txn_date STRING,
  filler STRING,
  txn_time STRING,
  store_id STRING,
  terminal_id STRING,
  txn_id STRING,
  cust_id STRING,
  payment_mode STRING,
  partner_bank STRING,
  amount_paid DECIMAL(9,2),
  bank_payable DECIMAL(9,2),
  customer_payable DECIMAL(9,2),
  currency_code STRING,
  txn_status STRING
'''


In [34]:
df = spark.read.format("csv").schema(raw_data_lake_schema).option('header', True).load(r'D:\mainframe_to_analytics_dev\databricks\catalog\bronze\BRONZE20260104.CSV')

In [35]:
df.show(10)

+-------+----------+------+--------+--------+-----------+------------+----------+------------+---------------+-----------+------------+----------------+-------------+----------+
|_row_id|  txn_date|filler|txn_time|store_id|terminal_id|      txn_id|   cust_id|payment_mode|   partner_bank|amount_paid|bank_payable|customer_payable|currency_code|txn_status|
+-------+----------+------+--------+--------+-----------+------------+----------+------------+---------------+-----------+------------+----------------+-------------+----------+
|      0|2026-01-04|      |12:47:16|  STR001|       T01 |TXN000001   |CUST89581 |  CASH      |NA             |    1988.85|     2038.57|         2018.68|          ESS|          |
|      1|2026-01-04|      |12:47:16|  STR002|       T01 |TXN000002   |CUST62506 |  UPI       |HDFC           |     716.64|      734.56|          727.39|          ED |          |
|      2|2026-01-04|      |12:47:16|  STR002|       T01 |TXN000003   |CUST55832 |  CARD      |HDFC           |

In [36]:
df.printSchema()

root
 |-- _row_id: long (nullable = true)
 |-- txn_date: string (nullable = true)
 |-- filler: string (nullable = true)
 |-- txn_time: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- terminal_id: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- cust_id: string (nullable = true)
 |-- payment_mode: string (nullable = true)
 |-- partner_bank: string (nullable = true)
 |-- amount_paid: decimal(9,2) (nullable = true)
 |-- bank_payable: decimal(9,2) (nullable = true)
 |-- customer_payable: decimal(9,2) (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- txn_status: string (nullable = true)



In [37]:
## Type Casting

# txn_date, txn_time -> datetime, timestamp


In [38]:
from pyspark.sql.functions import to_timestamp, concat_ws, col

df = df.withColumn(
    "txn_timestamp",
    to_timestamp(
        concat_ws(" ", col("txn_date"), col("txn_time")),
        "yyyy-MM-dd HH:mm:ss"
    )
)

In [39]:
df.show(10)

+-------+----------+------+--------+--------+-----------+------------+----------+------------+---------------+-----------+------------+----------------+-------------+----------+-------------------+
|_row_id|  txn_date|filler|txn_time|store_id|terminal_id|      txn_id|   cust_id|payment_mode|   partner_bank|amount_paid|bank_payable|customer_payable|currency_code|txn_status|      txn_timestamp|
+-------+----------+------+--------+--------+-----------+------------+----------+------------+---------------+-----------+------------+----------------+-------------+----------+-------------------+
|      0|2026-01-04|      |12:47:16|  STR001|       T01 |TXN000001   |CUST89581 |  CASH      |NA             |    1988.85|     2038.57|         2018.68|          ESS|          |2026-01-04 12:47:16|
|      1|2026-01-04|      |12:47:16|  STR002|       T01 |TXN000002   |CUST62506 |  UPI       |HDFC           |     716.64|      734.56|          727.39|          ED |          |2026-01-04 12:47:16|
|      2|2

In [42]:
### Normalize values

## eg 'UPI   ' to 'UPI' 



In [52]:
from pyspark.sql.functions import trim, col

df = df.withColumn('payment_mode', trim(col('payment_mode')))
df = df.withColumn('partner_bank', trim(col('partner_bank')))
df = df.withColumn('currency_code', trim(col('currency_code')))
df = df.withColumn('terminal_id', trim(col('terminal_id')))
df.show(10)

+-------+----------+------+--------+--------+-----------+------------+----------+------------+------------+-----------+------------+----------------+-------------+----------+-------------------+-------+-------------+
|_row_id|  txn_date|filler|txn_time|store_id|terminal_id|      txn_id|   cust_id|payment_mode|partner_bank|amount_paid|bank_payable|customer_payable|currency_code|txn_status|      txn_timestamp|IS_CASH|is_self_kiosk|
+-------+----------+------+--------+--------+-----------+------------+----------+------------+------------+-----------+------------+----------------+-------------+----------+-------------------+-------+-------------+
|      0|2026-01-04|      |12:47:16|  STR001|        T01|TXN000001   |CUST89581 |        CASH|          NA|    1988.85|     2038.57|         2018.68|          ESS|          |2026-01-04 12:47:16|      Y|            Y|
|      1|2026-01-04|      |12:47:16|  STR002|        T01|TXN000002   |CUST62506 |         UPI|        HDFC|     716.64|      734.56|

In [53]:
df = df.withColumn('IS_CASH', when(((col('payment_mode') == 'UPI') | (col('payment_mode') == 'CARD')),'N').otherwise('Y'))

In [54]:
df.show(10)

+-------+----------+------+--------+--------+-----------+------------+----------+------------+------------+-----------+------------+----------------+-------------+----------+-------------------+-------+-------------+
|_row_id|  txn_date|filler|txn_time|store_id|terminal_id|      txn_id|   cust_id|payment_mode|partner_bank|amount_paid|bank_payable|customer_payable|currency_code|txn_status|      txn_timestamp|IS_CASH|is_self_kiosk|
+-------+----------+------+--------+--------+-----------+------------+----------+------------+------------+-----------+------------+----------------+-------------+----------+-------------------+-------+-------------+
|      0|2026-01-04|      |12:47:16|  STR001|        T01|TXN000001   |CUST89581 |        CASH|          NA|    1988.85|     2038.57|         2018.68|          ESS|          |2026-01-04 12:47:16|      Y|            Y|
|      1|2026-01-04|      |12:47:16|  STR002|        T01|TXN000002   |CUST62506 |         UPI|        HDFC|     716.64|      734.56|

In [55]:
from pyspark.sql.functions import when, col

df = df.withColumn(
    "is_self_kiosk",
    when(col("terminal_id").isin("T01", "T02"), "N")
    .otherwise("Y")
)

In [56]:
df.show(10)

+-------+----------+------+--------+--------+-----------+------------+----------+------------+------------+-----------+------------+----------------+-------------+----------+-------------------+-------+-------------+
|_row_id|  txn_date|filler|txn_time|store_id|terminal_id|      txn_id|   cust_id|payment_mode|partner_bank|amount_paid|bank_payable|customer_payable|currency_code|txn_status|      txn_timestamp|IS_CASH|is_self_kiosk|
+-------+----------+------+--------+--------+-----------+------------+----------+------------+------------+-----------+------------+----------------+-------------+----------+-------------------+-------+-------------+
|      0|2026-01-04|      |12:47:16|  STR001|        T01|TXN000001   |CUST89581 |        CASH|          NA|    1988.85|     2038.57|         2018.68|          ESS|          |2026-01-04 12:47:16|      Y|            N|
|      1|2026-01-04|      |12:47:16|  STR002|        T01|TXN000002   |CUST62506 |         UPI|        HDFC|     716.64|      734.56|

In [60]:
silver_df = df

In [ ]:
## STEP 3 : BRONZE CSV FILE -> CATALOG

## WILL WORK ON DATABRICKS
import os
from datetime import datetime
SILVER_DIR = r"D:\mainframe_to_analytics_dev\databricks\catalog\silver"
run_date = datetime.now().strftime("%Y%m%d")
BRONZE_FILE = os.path.join(
            SILVER_DIR, f"BRONZE{run_date}.CSV"
        )

os.makedirs(SILVER_DIR, exist_ok=True)

silver_df.to_csv(BRONZE_FILE)

Py4JJavaError: An error occurred while calling o331.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: